In [1]:
import torch
import os
import random
import pandas as pd
import torch.nn as nn
import torch.nn.functional as F

from tx1_pre_process import *
from old_preprocess import *

In [2]:
DATA_PATH = '/home/ralleking/Downloads/nes_txt/'

In [3]:
def read_file(fname):
    with open(fname) as f:
        content = f.read().splitlines()
        content = [event.rsplit('_', 1) for event in content]
        content.append(['END', '0'])
    return content

def to_one_hot(vocab, event):
    one_hot = np.zeros(len(vocab))
    index = vocab[event]
    one_hot[index] = 1
    return one_hot

def padded_one_hot_time(vocab, time, pad_len):
    one_hot = np.zeros(pad_len+1)
    one_hot[time] = 1
    return one_hot

def from_one_hot(vocab, vector):
    ind = np.argmax(vector)
    for k, v in vocab.items():
        if v == ind:
            return k

def one_hot_tensor(pred):
    one_hot = torch.zeros(pred.shape)
    one_hot[:,pred.max(1)[1]] = 1
    return one_hot.double().to(device)

In [4]:
songs = []
for filename in os.listdir(DATA_PATH):
    file = read_file(DATA_PATH + filename)
    songs.extend(file) 

In [5]:
df = pd.DataFrame(songs, columns=['event', 'time'])
df['note_on'] = df.time.apply(lambda x: x.isdigit())
df['time'] = pd.to_numeric(df['time'].replace('NOTEOFF',0))
df.loc[df.note_on == False, 'event'] = df.event.apply(lambda x: x + '_NOTEOFF')
df.loc[df.event == 'END_NOTEOFF', 'event'] = 'END'

In [6]:
vocab = {}
i = 0
for item in df['event'].unique():    
    vocab[item] = i
    i += 1
    
df['one_hot_x'] = df.event.apply(lambda x: to_one_hot(vocab, x))
df['label'] = df.one_hot_x.apply(lambda x: np.argmax(x))

In [7]:
# get note on data
df.loc[df.event == 'WT', 'note_on'] = False
dfNote = df.loc[df.note_on == True]
dfNote = dfNote[dfNote.time != 0]
dfNote['note_class'] = dfNote['time'].apply(lambda x: padded_one_hot_time(vocab, x, 108))

In [8]:
data = df.values
data2 = dfNote.values

In [9]:
data_pipe4_x = []
data_pipe4_y = []

for i in range(0, len(data2)):
    x = np.asarray(data2[i,5].tolist())
    y = np.asarray(data2[i,1])

    x = torch.from_numpy(x).double()
    y = torch.from_numpy(y).double()
    
    data_pipe4_x.append(x)
    data_pipe4_y.append(y)
    if i>60000:
        break

In [10]:
start = 0
chunk_size = 10

data_pipe1_x = []
data_pipe1_y = []

data_pipe2_x = []
data_pipe2_y = []

data_pipe3_x = []
data_pipe3_y = []

num_songs = 0
for i in range(0, len(data)):    
    chunk_x = data[start:chunk_size]
    chunk_y = data[chunk_size:chunk_size+1]
    start = chunk_size + 1
    chunk_size = chunk_size + 11
    if np.in1d('END', chunk_x).all():
        num_songs +=1
        continue
    if np.in1d('END', chunk_y).all():
        num_songs +=1
        continue
    
    #Add correct columns to correct datapipes
    x = chunk_x[:,3].tolist()
    x = np.asarray(x)
    y = np.asarray(chunk_y[0,4])
    data_pipe1_x.append(torch.from_numpy(x).double())
    data_pipe1_y.append(torch.from_numpy(y).double())
    
    #pipe 2
    x2 = chunk_x[:,[1,4]].tolist()
    x2 = np.asarray(x2)
    y2 = np.asarray(chunk_y[0,1])
    data_pipe2_x.append(torch.from_numpy(x2).double())
    data_pipe2_y.append(torch.from_numpy(y2).double())

    #pipe 3
    x3 = np.asarray(data[i,3].tolist())
    y3 = data[i,1]
    
    data_pipe3_x.append(torch.from_numpy(x3).double())
    data_pipe3_y.append(torch.from_numpy(np.asarray(y3)).double())
    
    #For convenience
    if i>60000:
        break
print("Totalt number of songs " + str(num_songs))




/home/ralleking/miniconda/envs/deepdiva/lib/python3.7/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


Totalt number of songs 367


In [11]:
x = torch.stack(data_pipe1_x)
y = torch.stack(data_pipe1_y)
x = x.double()
y = y.double()

x2 = torch.stack(data_pipe2_x)
y2 = torch.stack(data_pipe2_y)
x2 = x2.double()
y2 = y2.double()

x3 = torch.stack(data_pipe3_x)
y3 = torch.stack(data_pipe3_y)
x3 = x3.double()
y3 = y3.double()

x4 = torch.stack(data_pipe4_x)
y4 = torch.stack(data_pipe4_y)
x4 = x4.double()
y4 = y4.double()


data_pipe1 = torch.utils.data.TensorDataset(x, y, x3, y3, x2)
data_pipe2 = torch.utils.data.TensorDataset(x4, y4)

In [12]:
#Has only sequence information. That is x = 10 previous one hot enchodings of sequences, y = the next one
batch_size = 200
data1 = torch.utils.data.DataLoader(data_pipe1, shuffle=True, batch_size=batch_size)
data2 = torch.utils.data.DataLoader(data_pipe2, shuffle=True, batch_size=batch_size)


In [13]:
# torch.cuda.is_available() checks and returns a Boolean True if a GPU is available, else it'll return False
is_cuda = torch.cuda.is_available()

# If we have a GPU available, we'll set our device to GPU. We'll use this device variable later in our code.
if is_cuda:
    device = torch.device("cuda")
    print("GPU is available")
else:
    device = torch.device("cpu")
    print("GPU not available, CPU used")

GPU is available


In [18]:
class Model(nn.Module):
    def __init__(self, input_size, output_size, hidden_dim, n_layers):
        super(Model, self).__init__()

        # Defining some parameters
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers

        #Defining the layers
        # RNN Layer
        self.rnn = nn.RNN(input_size, hidden_dim, n_layers, batch_first=True)   
        # Fully connected layer
        self.fc = nn.Linear(hidden_dim, output_size)
        self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        
        batch_size = x.size(0)
        
        hidden = self.init_hidden(batch_size)
        
        out, hidden = self.rnn(x, hidden)
        
        out = out.contiguous().view(-1, self.hidden_dim)
        out = self.dropout(out)
        out = self.fc(out)
        out = out.view(batch_size, 10, -1)
        out = out[:,-1]

        return out, hidden
    
    def init_hidden(self, batch_size):
        hidden = torch.zeros(self.n_layers, batch_size, self.hidden_dim).to(device).double()
        return hidden

class Model2(nn.Module):
    def __init__(self, hidden_dim, output_size, hidden_out):
        super(Model2, self).__init__()
        
        self.hidden_dim = hidden_dim
        self.hidden_out = hidden_out
        self.output_size = output_size
        
        self.hidden = nn.Linear(hidden_dim, hidden_out)
        self.output_layer = nn.Linear(hidden_out, 1)
        self.do = nn.Dropout(0.2)
        
    def forward(self, x):
        x = x.view(-1, self.hidden_dim)
        x = self.hidden(x)
        x = self.do(x)
        x = F.relu(self.output_layer(x))
        return x
    
class Model3(nn.Module):
    def __init__(self, input_size, output_size=109, hidden_dim=256):
        super(Model3, self).__init__()
        
        self.output_size = output_size
        self.input_size = input_size
        self.hidden_dim = hidden_dim
        
        self.hidden = nn.Linear(input_size, hidden_dim)
        self.output = nn.Linear(hidden_dim, output_size)
        self.do = nn.Dropout(0.25)
    
    def forward(self, x):
        x = x.view(-1, self.input_size)
        x = self.hidden(x)
        x = self.do(x)
        x = F.relu(x)
        x = self.output(x)
        x = F.softmax(x, dim=1)
        return x


In [24]:
# first model
model = Model(input_size=10, output_size=10, hidden_dim=10, n_layers=10)
model = model.to(device)
model = model.double()

n_epochs = 100
lr=0.0001

loss1 = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)


In [25]:
data_for_model3 = []

def perpare_for_model3(context, note, time):
    time = time.unsqueeze(1)
    a = torch.stack((context[-1], note), dim=1)
    b = a[a[:, 1, 0] != 1]
    #padding = torch.zeros((batch_size-b.shape[0], b.shape[1], b.shape[2])).double().to(device)
    #b = torch.cat((b, padding), dim=0)
    
    time = time[a[:, 1, 0] != 1]
    #padding_y = torch.zeros((batch_size-time.shape[0],1)).double().to(device)
    #time = torch.cat((time, padding_y), dim=0)
    
    out = (b.detach(), time.detach())
    return out

for epoch in range(1,n_epochs+1):
    for x,y, x3, y3,_ in data1:
        optimizer.zero_grad() # Clears existing gradients from previous epoch
        x = x.to(device)
        y = y.to(device)
        x3 = x3.to(device)
        y3 = y3.to(device)
        output, hidden = model(x)
        if epoch == n_epochs:
            d = perpare_for_model3(hidden, x3, y3)
            data_for_model3.append(d)
        loss_ce = loss1(output.squeeze(), y.long())
        loss_ce.backward()
        optimizer.step()
    if not epoch%10:
        print('Epoch: {}/{}.............'.format(epoch, n_epochs), end=' ')
        print("Loss: {:.4f}".format(loss_ce.item()))        

Epoch: 10/100............. Loss: 1.1255
Epoch: 20/100............. Loss: 1.0411
Epoch: 30/100............. Loss: 1.1971
Epoch: 40/100............. Loss: 0.9888
Epoch: 50/100............. Loss: 1.3047
Epoch: 60/100............. Loss: 1.1471
Epoch: 70/100............. Loss: 0.8749
Epoch: 80/100............. Loss: 0.9533
Epoch: 90/100............. Loss: 1.1256
Epoch: 100/100............. Loss: 0.6912


In [35]:
# third model (note model)
model3 = Model3(20)
model3 = model3.to(device)
model3 = model3.double()

n_epochs = 100
lr=0.001

loss3 = nn.CrossEntropyLoss()
optimizer3 = torch.optim.Adam(model3.parameters(), lr=lr)

In [36]:
for epoch in range(1,n_epochs+1):
    for x, y in data_for_model3:
        optimizer3.zero_grad() # Clears existing gradients from previous epoch        
        x = x.to(device)
        y = y.to(device)
        output = model3(x)
        loss_ce3 = loss3(output, y.long().squeeze())
        loss_ce3.backward()
        optimizer3.step()
    if not epoch%10:
        print('Epoch: {}/{}.............'.format(epoch, n_epochs), end=' ')
        print("Loss: {:.4f}".format(loss_ce3.item()))

Epoch: 10/100............. Loss: 4.5097
Epoch: 20/100............. Loss: 4.5288
Epoch: 30/100............. Loss: 4.5255
Epoch: 40/100............. Loss: 4.5095
Epoch: 50/100............. Loss: 4.4915
Epoch: 60/100............. Loss: 4.4833
Epoch: 70/100............. Loss: 4.4815
Epoch: 80/100............. Loss: 4.4740
Epoch: 90/100............. Loss: 4.4724
Epoch: 100/100............. Loss: 4.4788


In [39]:
#second model
model2 = Model2(20, 1, 10)
model2 = model2.to(device)
model2 = model2.double()

n_epochs = 10

lr2=0.01
loss2 = nn.MSELoss()
optimizer2 = torch.optim.Adam(model2.parameters(), lr=lr)

In [ ]:
for epoch in range(1,n_epochs+1):
    for x, y, x3, y3,_ in data1:
        optimizer2.zero_grad()
        x3 = x3.to(device)
        y3 = y3.to(device)
        x = x.to(device)
        y = y.to(device)
        
        prediction, _ = model(x)
        X = shape_data(prediction, x3)
        output = model2(X)
        loss_mse = loss2(output, y3)

        loss_mse.backward()
        optimizer2.step()
    if not epoch%10:   
        print('Epoch: {}/{}.............'.format(epoch, n_epochs), end=' ')
        print("Loss: {:.4f}".format(loss_mse.item()))

In [ ]:
def shape_data(from_first_model, x2):
    X = torch.cat((from_first_model, x2), dim=1).to(device)
    return X

#take tensor of shape (1x10x10)
def predict_seq(sample, model, pred_len, model2, y2, model3, time):
    starting_times = list(y2[:,0].squeeze())
    notes = sample
    model = model.to(device)
    sample = sample.to(device)
    note = start_note
    for i in range(0,pred_len):
        prediction = model(sample)[0]
        X = shape_data(prediction, note)
        
        time_pred = model2(X)     
        prediction = one_hot_tensor(prediction)
        
        if prediction[0][0] != 1:
            input_model3 = perpare_for_model3(context, prediction, time)            
            p = model3(input_model3[0])
            max_val, max_index = torch.max(p, 1)
            starting_times.append(max_index)
        else:
            starting_times.append(time_pred)
        
        prediction = prediction.view(1,1,10)
        notes = torch.cat((notes, prediction), dim=1)
        sample = torch.cat((sample, prediction), dim=1)  
        sample = sample[:,1:,:]
    return notes, starting_times

def prediction_to_numpy(predicted_sequence, times):
    unpacked_times = [str(int(np.squeeze(time.to("cpu").detach().numpy()))) for time in times]
    song = predicted_sequence.to("cpu")
    song = song.squeeze()
    song = song.numpy()
    return song, unpacked_times


def numpy_to_txt(song, vocab, times):
    i = 0
    new_song = []
    song = song.tolist()
    for event in song:
        note = from_one_hot(vocab, event)
        note = note + "_" + times[i]
        new_song.append(note)
        i += 1
    return new_song

def clean_output(txt_song):
    clean_song = []
    for note in txt_song:
        note = note.rsplit('_',1)
        if 'OFF' in note[0]:
            clean_song.append(note[0])
        else:
            clean_song.append('_'.join(note))
    return clean_song

In [ ]:
sample_no = 6
data = iter(data1)
sample = next(data)

x = sample[0][sample_no].view(1,10,10).to(device)
start_note = sample[2][sample_no].view(1,10).to(device)
time = sample[3][sample_no].view(1).to(device)
starting_times = sample[4][sample_no].to(device)
first_pred, context = model(x)


In [ ]:
test, times = predict_seq(x, model, 300, model2, starting_times, model3, time)
np_song, times = prediction_to_numpy(test, times)
txt_song = numpy_to_txt(np_song, vocab, times)
s = clean_output(txt_song)
txt_song

In [ ]:
np.savetxt('s.txt', np.asarray(s), delimiter=',', fmt='%s')